# Bayesian inference of average causal effect with instrumental variables

In [ ]:
import stan
import numpy as np
from scipy.stats import beta
import matplotlib.pyplot as plt

We will implement inference of average causal effect using the finite-response treatment given by Pearl, Causality 2009.

## Warm-up: using PyStan & beta-binomial model

Below is a minimal working example for using PyStan for inference on a coin flip with a beta-distributed prior.

In [ ]:
x = np.random.choice([0,1], p=[0.2,0.8], size=1000) # data

Declare the model using the Stan probabilistic programming language

In [ ]:
coin_flip = """
data {
  int<lower=0> N;
  int<lower=0, upper=1> x[N];   
}
parameters {
  real<lower=0, upper=1> theta;
}
model {
  theta ~ beta(0.5, 0.5);
  x ~ bernoulli(theta);
}
"""

Compile the model

In [ ]:
coin_data = {"x": x, "N":len(x)}

In [ ]:
posterior = stan.build(coin_flip, data=coin_data, random_seed=1)

Perform inference

In [ ]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Compare to theory

In [ ]:
df = fit.to_frame()

In [ ]:
a = 0.5 + np.sum(x)
b = 0.5 + len(x) - np.sum(x)
theta_sp = np.linspace(0.79, 0.86)
posterior_theory = beta.pdf(theta_sp, a, b)

In [ ]:
fig, ax = plt.subplots()
df['theta'].hist(ax=ax, bins='auto', label='samples', density=True)
ax.plot(theta_sp, posterior_theory, '-k', label='theory')
ax.set_xlabel(r'$\theta$')
ax.set_ylabel(r'$P(\theta)$')
ax.legend()

As expected

## Finite-response treatment of instrumental variable model

Imagine we are running a clinical trial, and the variable $Z \in \{0,1\}$ represents the treatment assigned to a patient. Let $X \in \{0,1\}$ denote whether the patient takes the treatment, and $Y \in \{0,1\}$ denote the observed response. There are factors (both observed and unobserved) $U$ that influence the way a subject responds to treatments, which may also affect the patient's choice to take the treatment. When $z \neq x$ we have imperfect compliance.

 <img src="Figures/instrumental-variable.jpg" style="width:500px"> 

Pearl (Causality, 2009) shows that regardless of the domain of $U$, it can always be partitioned into four equivalence classes for the relationship between $X$ and $Y$, and similar for $Z$ and $X$, resulting in a 16 possible subpopulations to describe $U$ completely:

 <img src="Figures/ace.jpg" style="width:400px"> 

Let's generate the data $X$, $Y$, $Z$ according to these canonical partitions, so we have access to the ground truth for inference

In [ ]:
from scipy.stats import multinomial, dirichlet

In [ ]:
p_subpop = dirichlet.rvs(0.5*np.ones(16), size=1, random_state=42)
p_subpop = p_subpop.squeeze(0)

In [ ]:
multinomial.rvs(1, p_subpop, size=N, random_state=42)

In [ ]:
N = 1000
R = multinomial.rvs(1, p_subpop, size=N, random_state=42).reshape((N,4,4))
_, rx, ry = np.where(R)

In [ ]:
R[0]

In [ ]:
rx[0], ry[0]

In [ ]:
def make_response_mapping(u: int, r: int):
    if r==0:
        return 0
    elif r==1 and u==0:
        return 0
    elif r==1 and u==1:
        return 1
    elif r==2 and u==0:
        return 1
    elif r==2 and u==1:
        return 0
    elif r==3:
        return 1
    else:
        raise ValueError

In [ ]:
_make_response_mapping = np.vectorize(make_response_mapping)

In [ ]:
x = _make_response_mapping(z, rx)
y = _make_response_mapping(x, ry)

In [ ]:
z.shape, x.shape, y.shape

## Make PyStan model

In [ ]:
canonical_partition_instrument = """
data {
  int<lower=0> N;
  int<lower=0, upper=1> x[N];
  int<lower=0, upper=1> y[N];
  int<lower=0, upper=1> z[N];
}
parameters {  
  vector<lower=0, upper=1>[16] vr;
  int<lower=1, upper=16> r;
}
model {
  r ~ categorical(vr);
  
  if((r == 1) || (r == 5) || (r==9) || (r==13))
    x=0;
  else if (((r == 2) || (r == 6) || (r==10) || (r==14)) && (z==0))
    x=0;
  else if (((r == 2) || (r == 6) || (r==10) || (r==14)) && (z==1))
    x=1;
  else if (((r == 3) || (r == 7) || (r==11) || (r==15)) && (z==0))
    x=1;
  else if (((r == 3) || (r == 7) || (r==11) || (r==15)) && (z==1))
    x=0;
  else if((r == 4) || (r == 8) || (r==12) || (r==16))
    x=1;

  if((r == 1) || (r == 2) || (r==3) || (r==4))
    y=0;
  else if (((r == 5) || (r == 6) || (r==7) || (r==8)) && (x==0))
    y=0;
  else if (((r == 5) || (r == 6) || (r==7) || (r==8)) && (x==1))
    y=1;
  else if (((r == 9) || (r == 10) || (r==11) || (r==12)) && (x==0))
    y=1;
  else if (((r == 9) || (r == 10) || (r==11) || (r==12)) && (x==1))
    y=0;
  else if (((r == 13) || (r == 14) || (r==15) || (r==16)) && (x==0))
    y=1;
      
}
"""


Compile the model

In [ ]:
canonical_partition_instrument_data = {"x": x, "y":y, "z":z, "N":N}

In [ ]:
posterior = stan.build(canonical_partition_instrument, data=canonical_partition_instrument_data, random_seed=1)

It turns out that latent discrete variables are not supported in Stan [see here](https://mc-stan.org/docs/2_22/stan-users-guide/latent-discrete-chapter.html). Would have to use a Gibbs sampler like BUGS or JAGS. Nevermind, it was worth a try.